In [1]:
import tensorflow as tf
import os

In [2]:
!pip list

Package                       Version
----------------------------- ---------------
absl-py                       2.1.0
aiobotocore                   2.5.0
aiofiles                      22.1.0
aiohttp                       3.8.5
aioitertools                  0.7.1
aiosignal                     1.2.0
aiosqlite                     0.18.0
alabaster                     0.7.12
anaconda-anon-usage           0.4.2
anaconda-catalogs             0.2.0
anaconda-client               1.12.1
anaconda-cloud-auth           0.1.3
anaconda-navigator            2.5.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs     

In [3]:
import cv2
import imghdr

C:\Users\Hp\AppData\Local\Temp\ipykernel_21496\4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [32]:
data_dir="Downloads/data"


In [33]:
image_extns=["jpeg","jpg","bmp","png"]

In [35]:
os.listdir(data_dir)

['happy.zip', 'sad.zip']

In [48]:

import zipfile
with zipfile.ZipFile("C:/Users/Hp/Downloads/data/sad.zip", 'r') as zip_ref:
    zip_ref.extractall("Downloads/data/sad")
  


In [51]:
os.listdir(data_dir)

['.ipynb_checkpoints', 'happy', 'sad']

In [57]:

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,image_class)):
                            image_path=os.path.join(data_dir,image_class,image) 
                            try:
                                img=cv2.imread(image_path)
                                tip=imghdr.what(image_path)
                                if tip not in image_extns:
                                    print("removed")
                                    os.remove(image_path)
                            except Exception as e:
                                print("issue with img {}".format(image_path))
        
  

In [58]:
import tensorflow as tf

In [73]:
data=tf.keras.utils.image_dataset_from_directory(data_dir)

Found 179 files belonging to 2 classes.


In [60]:
import numpy as np

In [61]:
data_iterator=data.as_numpy_iterator()

In [62]:
batch=data_iterator.next()

In [69]:
scaled=batch[0]/255

In [75]:
data=data.map(lambda x,y:(x/255,y))

In [76]:
data.as_numpy_iterator().next()

(array([[[[1.        , 0.6745098 , 0.7529412 ],
          [0.9979779 , 0.6684436 , 0.746875  ],
          [0.98878676, 0.6408701 , 0.71930146],
          ...,
          [0.9923407 , 0.62371325, 0.68645835],
          [0.9954044 , 0.62677693, 0.6895221 ],
          [0.99607843, 0.627451  , 0.6901961 ]],
 
         [[0.9973039 , 0.6677696 , 0.746201  ],
          [0.9956873 , 0.6627461 , 0.74117744],
          [0.9883392 , 0.6399117 , 0.7183431 ],
          ...,
          [0.9919879 , 0.62371325, 0.6862819 ],
          [0.9947883 , 0.62677693, 0.689214  ],
          [0.9954044 , 0.627451  , 0.6898591 ]],
 
         [[0.985049  , 0.63713235, 0.7155637 ],
          [0.98527545, 0.636848  , 0.7152794 ],
          [0.98630464, 0.6355555 , 0.7139869 ],
          ...,
          [0.9903842 , 0.62371325, 0.6854801 ],
          [0.9919879 , 0.62677693, 0.6878138 ],
          [0.9923407 , 0.627451  , 0.6883272 ]],
 
         ...,
 
         [[0.7266238 , 0.71878064, 0.7619179 ],
          [0.73440

In [78]:
train_size=int(len(data)*.7)

In [79]:
train_size

4

In [81]:
val_size=int(len(data)*.2)

In [86]:
test_size=int(len(data)*.1)+1

In [87]:
train=data.take(train_size)
val=data.skip(train_size).take(val_size)
test=data.skip(train_size+val_size).take(test_size)

In [88]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

In [89]:
model=Sequential()

In [93]:
model.add(Conv2D(32,(3,3),1,activation="relu",input_shape=(256,256,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3),1,activation="relu",input_shape=(256,256,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32,(3,3),1,activation="relu",input_shape=(256,256,3)))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(256,activation="relu")),
model.add(Dense(1,activation="sigmoid"))

          

In [94]:
model.compile(optimizer="adam",loss=tf.losses.BinaryCrossentropy(),metrics="accuracy")

In [95]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 32)        18464     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 32)        0

In [97]:
logdir="logs"
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [103]:
history=model.fit(train,epochs=20,validation_data=val,callbacks=[tensorboard_callback])

Epoch 1/20


4/4 [==============================] - 7s 1s/step - loss: 1.6571 - accuracy: 0.5312 - val_loss: 0.6700 - val_accuracy: 0.6875
Epoch 2/20
4/4 [==============================] - 3s 607ms/step - loss: 0.7501 - accuracy: 0.5312 - val_loss: 0.5595 - val_accuracy: 0.7812
Epoch 3/20
4/4 [==============================] - 3s 654ms/step - loss: 0.6449 - accuracy: 0.6250 - val_loss: 0.5703 - val_accuracy: 0.7188
Epoch 4/20
4/4 [==============================] - 3s 607ms/step - loss: 0.5660 - accuracy: 0.7109 - val_loss: 0.3728 - val_accuracy: 0.9375
Epoch 5/20
4/4 [==============================] - 3s 593ms/step - loss: 0.6198 - accuracy: 0.7031 - val_loss: 0.7572 - val_accuracy: 0.6250
Epoch 6/20
4/4 [==============================] - 3s 598ms/step - loss: 0.5744 - accuracy: 0.7031 - val_loss: 0.4295 - val_accuracy: 0.7812
Epoch 7/20
4/4 [==============================] - 3s 630ms/step - loss: 0.4527 - accuracy: 0.7891 - val_loss: 0.4430 - val_accuracy: 0.8750
Epoch 8/20
4/4 [=====

In [104]:
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy

In [105]:
pre=Precision()
re=Recall()
acc=BinaryAccuracy()

In [106]:
for batch in test.as_numpy_iterator():
    X,y=batch
    yhat=model.predict(X)
    pre.update_state(y,yhat)
    re.update_state(y,yhat)
    acc.update_state(y,yhat)

1/1 [==============================] - 1s 642ms/step


In [107]:
print(pre.result())

tf.Tensor(1.0, shape=(), dtype=float32)


In [145]:
def ans(yhat):
    if yhat<.5:
        print("happy")
    else:
        print("sa")

In [158]:
from tensorflow.keras.models import load_model

In [157]:
model.save(os.path.join("models","H.S.IMAGECLASSIFIER.h5"))